In [1]:
!pip install razdel networkx pymorphy2[fast] nltk rouge==0.3.1
#!pip install --upgrade datasets tqdm transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 8.2 MB 12.4 MB/s 
     |████████████████████████████████| 371 kB 61.1 MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=847807 sha256=afb478d514bf22178e7ac98410c5fcc509e28fa3d940a0699a32e05154670d81
  Stored in directory: /root/.cache/pip/wheels/85/51/a4/2de41ff197786537075027c27b479a38da92f50abc86634445
Successfully built DAWG


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 35.7 MB/s 
     |████████████████████████████████| 1.1 MB 41.1 MB/s 
     |████████████████████████████████| 140 kB 73.0 MB/s 
     |████████████████████████████████| 101 kB 7.9 MB/s 
     |████████████████████████████████| 212 kB 13.0 MB/s 
     |████████████████████████████████| 596 kB 20.9 MB/s 
     |████████████████████████████████| 127 kB 78.3 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 144 kB 66.0 MB/s 
     |████████████████████████████████| 271 kB 7.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's depe

## Обучим модель на задачу суммаризации

In [4]:
from datasets import load_dataset

dataset_train = load_dataset('IlyaGusev/gazeta', revision="v1.0", split= 'train[:10%]')
dataset_test = load_dataset('IlyaGusev/gazeta', revision="v1.0", split= 'test[:10%]')

No config specified, defaulting to: gazeta/default


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/52400 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5770 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5265 [00:00<?, ? examples/s]

Dataset gazeta downloaded and prepared to /root/.cache/huggingface/datasets/IlyaGusev___gazeta/default/1.0.0/ef9349c3c0f3112ca4036520d76c4bc1b8a79d30bc29643c6cae5a094d44e457. Subsequent calls will reuse this data.


No config specified, defaulting to: gazeta/default
Reusing dataset gazeta (/root/.cache/huggingface/datasets/IlyaGusev___gazeta/default/1.0.0/ef9349c3c0f3112ca4036520d76c4bc1b8a79d30bc29643c6cae5a094d44e457)


In [ ]:
dataset_train

In [ ]:
dataset_test

In [ ]:
dataset_test['summary'][0]

In [ ]:
dataset_test['title'][0]

In [5]:
model_name = "IlyaGusev/mbart_ru_sum_gazeta"

In [6]:
def len_tok(text):
    return len(text.split())

In [ ]:
max_len_sum, max_len_tl = max(map(len_tok, dataset_train['summary'])), max(map(len_tok, dataset_train['title']))
max_len_sum, max_len_tl

In [7]:
max_len_sum, max_len_tl = 60, 15

In [8]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 28.6 MB/s 
     |████████████████████████████████| 1.2 MB 63.9 MB/s 
     |████████████████████████████████| 6.6 MB 59.0 MB/s 


In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    tokenized_input = tokenizer(batch['summary'], padding='max_length', truncation=True, max_length=max_len_sum)
    tokenized_label = tokenizer(batch['title'], padding='max_length', truncation=True, max_length=max_len_tl)

    tokenized_input['labels'] = tokenized_label['input_ids']

    return tokenized_input

dataset_train = dataset_train.map(tokenize, batched=True, batch_size=2)
dataset_test = dataset_test.map(tokenize, batched=True, batch_size=2)

#dataset_train.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])
#dataset_test.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])

loading configuration file https://huggingface.co/IlyaGusev/mbart_ru_sum_gazeta/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3d0575815a98e6b1e9cb0a9b8167b6e931e77cdf986fbd38c331b2ef3f362d1e.93a7d19d0600d7e712e8951dfca0313652cd1868e9ae8a7f5861f66fb22eee2b
Model config MBartConfig {
  "_name_or_path": "IlyaGusev/mbart_ru_sum_gazeta",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "MBartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 250021,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_

  0%|          | 0/2620 [00:00<?, ?ba/s]

  0%|          | 0/289 [00:00<?, ?ba/s]

In [16]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = AutoModelForSeq2SeqLM.from_pretrained(model_name) #, from_pt=True)

loading configuration file https://huggingface.co/IlyaGusev/mbart_ru_sum_gazeta/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3d0575815a98e6b1e9cb0a9b8167b6e931e77cdf986fbd38c331b2ef3f362d1e.93a7d19d0600d7e712e8951dfca0313652cd1868e9ae8a7f5861f66fb22eee2b
Model config MBartConfig {
  "_name_or_path": "IlyaGusev/mbart_ru_sum_gazeta",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "MBartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 250021,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_

In [ ]:
#data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [17]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
samples = dataset_train[:4]
samples = {k: v for k, v in samples.items() if k in ["attention_mask", "input_ids", "labels"]}

print("\n\n\n... LENGTH OF SAMPLES IN BATCH")
print("\t", [len(x) for x in samples["input_ids"]])

batch = data_collator(samples)
print("\n\n... SHAPE OF PYTORCH ARRAYS IN BATCH AFTER COLLATING")
for k,v in batch.items(): print(f"\t{k:<15} ----> {v.shape}")




... LENGTH OF SAMPLES IN BATCH
	 [60, 60, 60, 60]


... SHAPE OF PYTORCH ARRAYS IN BATCH AFTER COLLATING
	input_ids       ----> torch.Size([4, 60])
	attention_mask  ----> torch.Size([4, 60])
	labels          ----> torch.Size([4, 15])


In [12]:
# See what we have...
print(f"\n\n\n... DATA COLLATOR ...\n\n{data_collator}")
print(f"\n\n\n... TOKENIZED DATASETS ...\n\n{dataset_train}")




... DATA COLLATOR ...

DataCollatorWithPadding(tokenizer=PreTrainedTokenizerFast(name_or_path='IlyaGusev/mbart_ru_sum_gazeta', vocab_size=250027, model_max_len=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>', 'additional_special_tokens': ['ar_AR', 'cs_CZ', 'de_DE', 'en_XX', 'es_XX', 'et_EE', 'fi_FI', 'fr_XX', 'gu_IN', 'hi_IN', 'it_IT', 'ja_XX', 'kk_KZ', 'ko_KR', 'lt_LT', 'lv_LV', 'my_MM', 'ne_NP', 'nl_XX', 'ro_RO', 'ru_RU', 'si_LK', 'tr_TR', 'vi_VN', 'zh_CN']}), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')



... TOKENIZED DATASETS ...

Dataset({
    features: ['text', 'summary', 'title', 'date', 'url', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 5240
})


In [18]:
from transformers import TrainingArguments, Trainer

In [19]:
training_args = TrainingArguments("test-trainer", 
                                  num_train_epochs=1,
                                  report_to="none") #report_to prevents wandbe
#model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
trainer = Trainer(
    model,
    training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()

PyTorch: setting up devices


RuntimeError: ignored

К сожалению у Гугл Коллаба не хватило памяти для дообучения этой модели.